In [ ]:
import numpy as np
import pandas as pd
import os
import cv2
import random
import shutil
import csv
import keras
from keras.models import Sequential
from keras.applications import InceptionV3
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D , Dropout, Flatten, Dense,Input
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Nadam
from keras.initializers import RandomNormal
from keras.applications.imagenet_utils import preprocess_input

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!nvidia-smi

Wed Dec  7 22:42:30 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    26W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
num_classes = 3
img_rows, img_cols = 256, 256
batch_size = 30
num_epochs = 20

In [ ]:
test_df = pd.read_csv("/content/drive/MyDrive/BioProject/all_test.csv")

In [ ]:
val_df=pd.read_csv("/content/drive/MyDrive/BioProject/All_data/val_data.csv")

In [ ]:
train_datagen = ImageDataGenerator( rescale = 1./255,   shear_range = 0.05, zoom_range = 0.05, horizontal_flip = True, vertical_flip = True, rotation_range = 360, width_shift_range = 0.05, height_shift_range = 0.05)
test_datagen = ImageDataGenerator(rescale=1./255)


In [ ]:
K.clear_session()

In [ ]:
model = Sequential()
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(img_rows, img_cols, 3))
model.add(base_model)
model_layers = base_model.output
flatten_layer = Flatten(name = 'flatten')(model_layers)
dropout_layer = Dropout(0.5)(flatten_layer)
output_layer = Dense(num_classes, activation='softmax',kernel_initializer=RandomNormal(mean=0.0, stddev=0.001))(dropout_layer)
model = Model(inputs=base_model.input, outputs=output_layer)
for layer in base_model.layers:
    layer.trainable = True
model.compile(optimizer=Nadam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


87910968/87910968 [==============================] - 0s 0us/step


In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 127, 127, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 127, 127, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                             

In [ ]:
model_name = 'InceptionV3'
filepath="/content/drive/MyDrive/BioProject/models1/%s-{epoch:02d}-{val_accuracy:.4f}.hdf5"%model_name
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
for epoch_num in range(1,9): 
  train_df =pd.read_csv("/content/drive/MyDrive/BioProject/All_data/Train_data/train{}.csv".format(epoch_num)) 
  train_generator = train_datagen.flow_from_dataframe(
        dataframe=train_df,
        directory='/content/drive/MyDrive/BioProject/All_data/All',
        x_col="filename",
        y_col="class",
        target_size=(img_rows, img_cols),
        batch_size=30,
        class_mode='categorical',shuffle=True)
  validation_generator = test_datagen.flow_from_dataframe(
          dataframe=vad_df,
          directory='/content/drive/MyDrive/BioProject/All_data/All',
          x_col="filename",
          y_col="class",
          target_size=(img_rows, img_cols),
          batch_size=30,
          class_mode='categorical',shuffle=True)
  model.fit_generator(train_generator, epochs=num_epochs,validation_data=validation_generator,shuffle=True,callbacks=callbacks_list) #start training

Found 1433 validated image filenames belonging to 3 classes.
Found 845 validated image filenames belonging to 3 classes.


<ipython-input-13-bb6f6a79f178>:19: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, epochs=num_epochs,validation_data=validation_generator,shuffle=True,callbacks=callbacks_list) #start training


Epoch 1/20
48/48 [==============================] - 556s 11s/step - loss: 0.7888 - accuracy: 0.6504 - val_loss: 0.5437 - val_accuracy: 0.8107
Epoch 2/20
48/48 [==============================] - 47s 962ms/step - loss: 0.5124 - accuracy: 0.8032 - val_loss: 0.7213 - val_accuracy: 0.7172
Epoch 3/20
48/48 [==============================] - 47s 975ms/step - loss: 0.4713 - accuracy: 0.8269 - val_loss: 0.4849 - val_accuracy: 0.8533
Epoch 4/20
48/48 [==============================] - 48s 989ms/step - loss: 0.3959 - accuracy: 0.8493 - val_loss: 0.3713 - val_accuracy: 0.8675
Epoch 5/20
48/48 [==============================] - 48s 991ms/step - loss: 0.3553 - accuracy: 0.8639 - val_loss: 0.3350 - val_accuracy: 0.8828
Epoch 6/20
48/48 [==============================] - 46s 961ms/step - loss: 0.3025 - accuracy: 0.8765 - val_loss: 0.5965 - val_accuracy: 0.7704
Epoch 7/20
48/48 [==============================] - 46s 948ms/step - loss: 0.2840 - accuracy: 0.9023 - val_loss: 0.3933 - val_accuracy: 0.8485


In [ ]:
reports={}
validation_generator = test_datagen.flow_from_dataframe(
        dataframe=test_df,
        directory='/content/drive/MyDrive/BioProject/All_data/All',
        x_col="filename",
        y_col="class",
        target_size=(256, 256),
        batch_size=20,
        class_mode='categorical',shuffle=True)

Found 7334 validated image filenames belonging to 3 classes.


In [ ]:
trained_models=[]
for r,d,f in os.walk('/content/drive/MyDrive/BioProject/models1'):
  for file in f:
    trained_models.append(os.path.join(r,file)) 



In [ ]:
print(trained_models)

['/content/drive/MyDrive/BioProject/models1/InceptionV3-02-0.9243.hdf5']


In [ ]:
for trained_model in trained_models:
  k.clear_session()
  net=keras.models.load_model(trained_model)
  covid_label= validation_generator.class_indices['COVID-19']
  pneu_label= validation_generator.class_indices['pneumonia']
  normal_label= validation_generator.class_indices['normal'] 
  tp=0 
  fp=0 
  anum=0 
  ###########
  wrong_covid=0
  correct_covid=0
  not_detected_covid=0
  covid_num=0
  ###########
  wrong_pneu=0
  correct_pneu=0
  not_detected_pneu=0
  pneu_num=0
  ############
  wrong_normal=0
  correct_normal=0
  not_detected_normal=0
  normal_num=0
  ##############
  wrong_covid_normal=0 
  wrong_covid_pneu=0   
  wrong_pneu_covid=0   
  wrong_pneu_normal=0 
  wrong_normal_pneu=0 
  wrong_normal_covid=0  
  ################
  for num,img_name in enumerate(validation_generator.filenames): 
    gt_ind=validation_generator.classes[num] 
    img=cv2.resize(cv2.imread('/content/drive/MyDrive/BioProject/All_data/All/'+img_name),(256,256)) 
    img=img.astype('float32') / 255.0 
    pred_ind=np.argmax(net.predict(np.expand_dims(img,axis=0))[0]) 
    anum+=1 

    if gt_ind==covid_label:
      covid_num+=1
    if gt_ind==pneu_label:
      pneu_num+=1
    if gt_ind==normal_label:
      normal_num+=1
    if gt_ind==covid_label and pred_ind==covid_label: 
      correct_covid+=1
    if gt_ind==covid_label and pred_ind!=covid_label:
      not_detected_covid+=1
      if pred_ind==pneu_label:
        wrong_covid_pneu+=1
      elif pred_ind==normal_label:
        wrong_covid_normal+=1
    if gt_ind!=covid_label and pred_ind==covid_label:
      wrong_covid+=1

    if gt_ind==normal_label and pred_ind==normal_label: 
      correct_normal+=1
    if gt_ind==normal_label and pred_ind!=normal_label:
      not_detected_normal+=1
      if pred_ind==pneu_label:
        wrong_normal_pneu+=1
      elif pred_ind==covid_label:
        wrong_normal_covid+=1
    if gt_ind!=normal_label and pred_ind==normal_label:
      wrong_normal+=1
    ###########################################
    if gt_ind==pneu_label and pred_ind==pneu_label: 
      correct_pneu+=1
    if gt_ind==pneu_label and pred_ind!=pneu_label:
      not_detected_pneu+=1
      if pred_ind==normal_label:
        wrong_pneu_normal+=1
      elif pred_ind==covid_label:
        wrong_pneu_covid+=1
    if gt_ind!=pneu_label and pred_ind==pneu_label:
      wrong_pneu+=1
    ###########################################

    if pred_ind==gt_ind:
      tp+=1
    else:
      fp+=1
  reports[trained_model]=[correct_covid,wrong_covid,not_detected_covid,
                   correct_pneu,wrong_pneu,not_detected_pneu,
                   correct_normal,wrong_normal,not_detected_normal,
                   covid_num,pneu_num,normal_num,tp,fp,wrong_covid_normal,wrong_covid_pneu,
                   wrong_pneu_covid,wrong_pneu_normal,wrong_normal_pneu,wrong_normal_covid]
  print(trained_model)
  print('correct_covid: ',correct_covid/covid_num,'%')
  print('wrong_covid: ',wrong_covid)
  print('not_detected_covid: ',not_detected_covid/covid_num,'%')

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 29ms/step
/content/drive/MyDrive/BioProject/models1/InceptionV3-02-0.9243.hdf5
correct_covid:  0.8666666666666667 %
wrong_covid:  38
not_detected_covid:  0.13333333333333333 %
###########################


In [ ]:
temp = 'Inceptionv3'

In [ ]:
with open('/content/drive/MyDrive/results/{}-fold{}.csv'.format(temp,1),newline='', mode='w') as csvfile: 
     csv_writer = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
     csv_writer.writerow(['correct_covid','wrong_covid','not_detected_covid',
                          'correct_pneu','wrong_pneu','not_detected_pneu',
                          'correct_normal','wrong_normal','not_detected_normal',
                          'covid_num','pneu_num','normal_num','tp','fp','wrong_covid_normal','wrong_covid_pneu',
                  'wrong_pneu_covid','wrong_pneu_normal','wrong_normal_pneu','wrong_normal_covid'])
     for key in reports.keys():
         row=[key]
         csv_writer.writerow(reports[key])